In [1]:
import pyspark
import pandas as pd
import numpy as np

In [2]:
spark = (pyspark.sql.SparkSession.builder
         .master('local[*]')
         .getOrCreate())

In [3]:
ratings = spark.read.json('data/ratings.json')
ratings.show(10)

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|     858|     4|9.56678732E8|   6040|
|    2384|     4|9.56678754E8|   6040|
|     593|     5|9.56678754E8|   6040|
|    1961|     4|9.56678777E8|   6040|
|    1419|     3|9.56678856E8|   6040|
|     213|     5|9.56678856E8|   6040|
|    3111|     5|9.56678856E8|   6040|
|     573|     4|9.56678856E8|   6040|
|    3505|     4|9.56678856E8|   6040|
|    1734|     2|9.56678881E8|   6040|
+--------+------+------------+-------+
only showing top 10 rows



In [4]:
movies_metadata = spark.read.csv('data/movies_metadata.csv')
movies_metadata.first()

Row(_c0='adult', _c1='belongs_to_collection', _c2='budget', _c3='genres', _c4='homepage', _c5='id', _c6='imdb_id', _c7='original_language', _c8='original_title', _c9='overview', _c10='popularity', _c11='poster_path', _c12='production_companies', _c13='production_countries', _c14='release_date', _c15='revenue', _c16='runtime', _c17='spoken_languages', _c18='status', _c19='tagline', _c20='title', _c21='video', _c22='vote_average', _c23='vote_count')

In [5]:
requests = spark.read.json('data/requests.json')
requests.show(10)

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|    2019|   NaN|9.56678777E8|   6040|
|     759|   NaN|9.56679248E8|   6040|
|    2858|   NaN|9.56679275E8|   6040|
|     246|   NaN|9.56679413E8|   6040|
|    1617|   NaN|9.56679473E8|   6040|
|    2324|   NaN|9.56679629E8|   6040|
|    1089|   NaN|9.56679796E8|   6040|
|    2804|   NaN|9.56680123E8|   6039|
|     933|   NaN| 9.5668027E8|   6039|
|    1304|   NaN|9.56680308E8|   6039|
+--------+------+------------+-------+
only showing top 10 rows



In [11]:
movies = spark.read.csv('data/movies.dat')
movies.first()

Row(_c0="1::Toy Story (1995)::Animation|Children's|Comedy")

In [19]:
users = (spark.read.load("data/users.dat", format="csv", sep=":", inferSchema="true").drop('_c1', '_c3', '_c5', '_c7'))
users = (users
         .withColumnRenamed(users.schema.names[0], 'userID')
         .withColumnRenamed(users.schema.names[1], 'gender')
         .withColumnRenamed(users.schema.names[2], 'age')
         .withColumnRenamed(users.schema.names[3], 'occupation')
         .withColumnRenamed(users.schema.names[4], 'zip'))
users.show(10)

+------+------+---+----------+-----+
|userID|gender|age|occupation|  zip|
+------+------+---+----------+-----+
|     1|     F|  1|        10|48067|
|     2|     M| 56|        16|70072|
|     3|     M| 25|        15|55117|
|     4|     M| 45|         7|02460|
|     5|     M| 25|        20|55455|
|     6|     F| 50|         9|55117|
|     7|     M| 35|         1|06810|
|     8|     M| 25|        12|11413|
|     9|     M| 25|        17|61614|
|    10|     F| 35|         1|95370|
+------+------+---+----------+-----+
only showing top 10 rows



In [22]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
userID        6040 non-null int32
gender        6040 non-null object
age           6040 non-null int32
occupation    6040 non-null int32
zip           6040 non-null object
dtypes: int32(3), object(2)
memory usage: 165.2+ KB


In [25]:
symmetrify = lambda x: '00000'[len(x):] + x
symmetrify('1111')

'01111'

In [20]:
users = users.toPandas()

In [21]:
for col in users.columns:
    print(f'{col.upper()}:')
    print(users[col].value_counts())
    print()

USERID:
2047    1
2724    1
2712    1
665     1
4763    1
2716    1
669     1
4767    1
2720    1
673     1
4771    1
677     1
661     1
4775    1
2728    1
681     1
4779    1
2732    1
685     1
4783    1
2736    1
689     1
4759    1
2708    1
4851    1
2692    1
2680    1
633     1
4731    1
2684    1
       ..
3423    1
5472    1
1378    1
3427    1
5476    1
1382    1
3403    1
1354    1
5448    1
3399    1
1326    1
3375    1
5424    1
1330    1
3379    1
5428    1
1334    1
3383    1
5432    1
1338    1
3387    1
5436    1
1342    1
3391    1
5440    1
1346    1
3395    1
5444    1
1350    1
2049    1
Name: userID, Length: 6040, dtype: int64

GENDER:
M    4331
F    1709
Name: gender, dtype: int64

AGE:
25    2096
35    1193
18    1103
45     550
50     496
56     380
1      222
Name: age, dtype: int64

OCCUPATION:
4     759
0     711
7     679
1     528
17    502
12    388
14    302
20    281
2     267
16    241
6     236
10    195
3     173
15    144
13    142
11    129
5    